In [2]:
import os
import torch
os.environ['TORCH'] = torch.__version__
print(torch.__version__)


1.13.1


In [3]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

In [4]:
class GATLayer(nn.Module):
    """
    Simple PyTorch Implementation of the Graph Attention layer.
    """
    def __init__(self):
        super(GATLayer, self).__init__()
      
    def forward(self, input, adj):
        print("")

In [5]:
in_features = 5
out_features = 2
nb_nodes = 3

W = nn.Parameter(torch.zeros(size=(in_features, out_features))) #xavier paramiter inizializator
nn.init.xavier_uniform_(W.data, gain=1.414)

input = torch.rand(nb_nodes,in_features) 


# linear transformation
h = torch.mm(input, W)
N = h.size()[0]

print(h.shape)

torch.Size([3, 2])


In [6]:
a = nn.Parameter(torch.zeros(size=(2*out_features, 1))) #xavier paramiter inizializator
nn.init.xavier_uniform_(a.data, gain=1.414)
print(a.shape)

leakyrelu = nn.LeakyReLU(0.2)

torch.Size([4, 1])


In [7]:
a_input = torch.cat([h.repeat(1, N).view(N * N, -1), h.repeat(N, 1)], dim=1).view(N, -1, 2 * out_features)

In [8]:
e = leakyrelu(torch.matmul(a_input, a).squeeze(2))

In [9]:
print(a_input.shape,a.shape)
print("")
print(torch.matmul(a_input,a).shape)
print("")
print(torch.matmul(a_input,a).squeeze(2).shape)

torch.Size([3, 3, 4]) torch.Size([4, 1])

torch.Size([3, 3, 1])

torch.Size([3, 3])


In [10]:
# Masked Attention
adj = torch.randint(2, (3, 3))

zero_vec  = -9e15*torch.ones_like(e)
print(zero_vec.shape)

torch.Size([3, 3])


In [11]:
attention = torch.where(adj > 0, e, zero_vec)
print(adj,"\n",e,"\n",zero_vec)
attention

tensor([[1, 1, 1],
        [0, 1, 1],
        [1, 1, 0]]) 
 tensor([[ 0.2799,  0.6508,  1.2550],
        [ 2.1092,  2.4801,  3.0843],
        [-0.0248,  0.2467,  0.8509]], grad_fn=<LeakyReluBackward0>) 
 tensor([[-9.0000e+15, -9.0000e+15, -9.0000e+15],
        [-9.0000e+15, -9.0000e+15, -9.0000e+15],
        [-9.0000e+15, -9.0000e+15, -9.0000e+15]])


tensor([[ 2.7986e-01,  6.5076e-01,  1.2550e+00],
        [-9.0000e+15,  2.4801e+00,  3.0843e+00],
        [-2.4841e-02,  2.4670e-01, -9.0000e+15]], grad_fn=<WhereBackward0>)

In [12]:
attention = F.softmax(attention, dim=1)
h_prime   = torch.matmul(attention, h)

In [13]:
attention

tensor([[0.1961, 0.2841, 0.5198],
        [0.0000, 0.3534, 0.6466],
        [0.4325, 0.5675, 0.0000]], grad_fn=<SoftmaxBackward0>)

In [14]:
h_prime

tensor([[-1.0315, -0.2346],
        [-1.1860, -0.3200],
        [-0.8687, -0.7757]], grad_fn=<MmBackward0>)

In [15]:
print(h_prime,"\n",h)

tensor([[-1.0315, -0.2346],
        [-1.1860, -0.3200],
        [-0.8687, -0.7757]], grad_fn=<MmBackward0>) 
 tensor([[-0.3982,  0.1155],
        [-1.2274, -1.4550],
        [-1.1634,  0.3003]], grad_fn=<MmBackward0>)


In [16]:
class GATLayer(nn.Module):
    def __init__(self, in_features, out_features, dropout, alpha, concat=True):
        super(GATLayer, self).__init__()
        
        '''
        TODO
        '''
        
    def forward(self, input, adj):
        # Linear Transformation
        h = torch.mm(input, self.W) # matrix multiplication
        N = h.size()[0]

        # Attention Mechanism
        a_input = torch.cat([h.repeat(1, N).view(N * N, -1), h.repeat(N, 1)], dim=1).view(N, -1, 2 * self.out_features)
        e       = self.leakyrelu(torch.matmul(a_input, self.a).squeeze(2))

        # Masked Attention
        zero_vec  = -9e15*torch.ones_like(e)
        attention = torch.where(adj > 0, e, zero_vec)
        
        attention = F.softmax(attention, dim=1)
        attention = F.dropout(attention, self.dropout, training=self.training)
        h_prime   = torch.matmul(attention, h)

        if self.concat:
            return F.elu(h_prime)
        else:
            return h_prime

In [17]:
class GATLayer(nn.Module):
    def __init__(self, in_features, out_features, dropout, alpha, concat=True):
        super(GATLayer, self).__init__()
        self.dropout       = dropout        # drop prob = 0.6
        self.in_features   = in_features    # 
        self.out_features  = out_features   # 
        self.alpha         = alpha          # LeakyReLU with negative input slope, alpha = 0.2
        self.concat        = concat         # conacat = True for all layers except the output layer.

        
        # Xavier Initialization of Weights
        # Alternatively use weights_init to apply weights of choice 
        self.W = nn.Parameter(torch.zeros(size=(in_features, out_features)))
        nn.init.xavier_uniform_(self.W.data, gain=1.414)
        
        self.a = nn.Parameter(torch.zeros(size=(2*out_features, 1)))
        nn.init.xavier_uniform_(self.a.data, gain=1.414)
        
        # LeakyReLU
        self.leakyrelu = nn.LeakyReLU(self.alpha)

    def forward(self, input, adj):
        # Linear Transformation
        h = torch.mm(input, self.W) # matrix multiplication
        N = h.size()[0]
        print(N)

        # Attention Mechanism
        a_input = torch.cat([h.repeat(1, N).view(N * N, -1), h.repeat(N, 1)], dim=1).view(N, -1, 2 * self.out_features)
        e       = self.leakyrelu(torch.matmul(a_input, self.a).squeeze(2))

        # Masked Attention
        zero_vec  = -9e15*torch.ones_like(e)
        attention = torch.where(adj > 0, e, zero_vec)
        
        attention = F.softmax(attention, dim=1)
        attention = F.dropout(attention, self.dropout, training=self.training)
        h_prime   = torch.matmul(attention, h)

        if self.concat:
            return F.elu(h_prime)
        else:
            return h_prime

In [21]:
from torch_geometric.data import Data
from torch_geometric.nn import GATConv
from torch_geometric.datasets import Planetoid
import torch_geometric.transforms as T

import matplotlib.pyplot as plt

name_data = 'Cora'
dataset = Planetoid(root= '/tmp/' + name_data, name = name_data)
dataset.transform = T.NormalizeFeatures()

print(f"Number of Classes in {name_data}:", dataset.num_classes)
print(f"Number of Node Features in {name_data}:", dataset.num_node_features)

Number of Classes in Cora: 7
Number of Node Features in Cora: 1433


Processing...
Done!


In [22]:
class GAT(torch.nn.Module):
    def __init__(self):
        super(GAT, self).__init__()
        self.hid = 8
        self.in_head = 8
        self.out_head = 1
        
        
        self.conv1 = GATConv(dataset.num_features, self.hid, heads=self.in_head, dropout=0.6)
        self.conv2 = GATConv(self.hid*self.in_head, dataset.num_classes, concat=False,
                             heads=self.out_head, dropout=0.6)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
                
        x = F.dropout(x, p=0.6, training=self.training)
        x = self.conv1(x, edge_index)
        x = F.elu(x)
        x = F.dropout(x, p=0.6, training=self.training)
        x = self.conv2(x, edge_index)
        
        return F.log_softmax(x, dim=1)
    
    
    
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = "cpu"

model = GAT().to(device)
data = dataset[0].to(device)


optimizer = torch.optim.Adam(model.parameters(), lr=0.005, weight_decay=5e-4)

model.train()
for epoch in range(1000):
    model.train()
    optimizer.zero_grad()
    out = model(data)
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
    
    if epoch%200 == 0:
        print(loss)
    
    loss.backward()
    optimizer.step()
    
    

tensor(1.9423, grad_fn=<NllLossBackward0>)
tensor(0.6565, grad_fn=<NllLossBackward0>)
tensor(0.6622, grad_fn=<NllLossBackward0>)
tensor(0.5666, grad_fn=<NllLossBackward0>)
tensor(0.4820, grad_fn=<NllLossBackward0>)


In [23]:
model.eval()
_, pred = model(data).max(dim=1)
correct = float(pred[data.test_mask].eq(data.y[data.test_mask]).sum().item())
acc = correct / data.test_mask.sum().item()
print('Accuracy: {:.4f}'.format(acc))

Accuracy: 0.8280


In [24]:
from torch_geometric_temporal.dataset import ChickenpoxDatasetLoader
from torch_geometric_temporal.signal import temporal_signal_split

loader = ChickenpoxDatasetLoader()

dataset = loader.get_dataset()

train_dataset, test_dataset = temporal_signal_split(dataset, train_ratio=0.2)

In [27]:
print(train_dataset)

In [28]:
import torch
import torch.nn.functional as F
from torch_geometric_temporal.nn.recurrent import DCRNN

class RecurrentGCN(torch.nn.Module):
    def __init__(self, node_features):
        super(RecurrentGCN, self).__init__()
        self.recurrent = DCRNN(node_features, 32, 1)
        self.linear = torch.nn.Linear(32, 1)

    def forward(self, x, edge_index, edge_weight):
        h = self.recurrent(x, edge_index, edge_weight)
        h = F.relu(h)
        h = self.linear(h)
        return h

In [29]:
from tqdm import tqdm

model = RecurrentGCN(node_features = 4)

optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

model.train()

for epoch in tqdm(range(200)):
    cost = 0
    for time, snapshot in enumerate(train_dataset):
        y_hat = model(snapshot.x, snapshot.edge_index, snapshot.edge_attr)
        cost = cost + torch.mean((y_hat-snapshot.y)**2)
    cost = cost / (time+1)
    cost.backward()
    optimizer.step()
    optimizer.zero_grad()

100%|██████████| 200/200 [00:36<00:00,  5.54it/s]


In [30]:
model.eval()
cost = 0
for time, snapshot in enumerate(test_dataset):
    y_hat = model(snapshot.x, snapshot.edge_index, snapshot.edge_attr)
    cost = cost + torch.mean((y_hat-snapshot.y)**2)
cost = cost / (time+1)
cost = cost.item()
print("MSE: {:.4f}".format(cost))


MSE: 1.0245


In [3]:
import pulp

# Problem data:

# Demands
demands = {
    "Chongqing": 16,
    "Beijing": 20,
    "Nanjing": 21,
    "Shijiazhuang": 12
}

# Distances to Ningbo
distances = {
    "Chongqing": 1750,
    "Beijing": 1400,
    "Nanjing": 450,
    "Shijiazhuang": 1300
}

# Bids in Yuan Renminbi per TEU per km
cost_per_TEU = {
    "Red": {
        "Chongqing": 7.50,
        "Beijing": 6.40,
        "Nanjing": 8.80,
        "Shijiazhuang": 7.80
    },
    "Blue": {
        "Chongqing": 9.00,
        "Beijing": 8.30,
        "Nanjing": 8.90,
        "Shijiazhuang": 9.20
    },
    "Green": {
        "Chongqing": 8.60,
        "Beijing": 8.00,
        "Nanjing": 9.20,
        "Shijiazhuang": 8.40
    }
}

# Carrier capacity
carrier_capacity = {
    "Red": 15,
    "Blue": 25,
    "Green": 90
}

bids = list(carrier_capacity.keys())

# LP formulation begins

# Define the LP problem
prob = pulp.LpProblem("CarrierAllocation", pulp.LpMinimize)

# Decision variables:
# x_{ij} - Binary variable that is 1 if lane i is assigned to carrier j, 0 otherwise.
x = pulp.LpVariable.dicts("x", (demands.keys(), bids), cat="Binary")

# Objective function:
# Minimize the sum of products of the cost per TEU per km, distance, expected weekly demand, and the decision variables.
prob += pulp.lpSum([x[city][carrier] * cost_per_TEU[carrier][city] * distances[city] * demands[city] for city in demands for carrier in bids])

# Constraints:

# 1. Each lane (city) is assigned to only one carrier.
for city in demands:
    prob += pulp.lpSum([x[city][carrier] for carrier in bids]) == 1

# 2. The total TEUs assigned to a carrier must not exceed its capacity.
for carrier in bids:
    prob += pulp.lpSum([x[city][carrier] * demands[city] for city in demands]) <= carrier_capacity[carrier]

# Solve the problem
prob.solve()

# Extract and display the results
allocated_TEUs = {carrier: sum(demands[city] * x[city][carrier].varValue for city in demands) for carrier in bids}
total_cost = sum(demands[city] * cost_per_TEU[carrier][city] * distances[city] * x[city][carrier].varValue for city in demands for carrier in bids)

print(allocated_TEUs)
print("Total cost:", round(total_cost))


{'Red': 12.0, 'Blue': 21.0, 'Green': 36.0}
Total cost: 670585


In [4]:
import pulp

# Define the LP problem
prob = pulp.LpProblem("CarrierAllocationWithNewConstraints", pulp.LpMinimize)

# Decision variables:
# x_{ij} - Continuous variable that represents the number of loads assigned from lane i to carrier j.
x = pulp.LpVariable.dicts("x", (demands.keys(), bids), lowBound=0, cat="Continuous")

# y_{ij} - Binary variable that equals 1 if carrier j is assigned at least 5 loads from lane i (otherwise 0).
y = pulp.LpVariable.dicts("y", (demands.keys(), bids), cat="Binary")

# Objective function:
# Minimize the sum of products of the cost per TEU per km, distance, expected weekly demand, and the decision variables.
prob += pulp.lpSum([x[city][carrier] * cost_per_TEU[carrier][city] * distances[city] for city in demands for carrier in bids])

# Constraints:

# 1. Each lane (city) demand must be satisfied.
for city in demands:
    prob += pulp.lpSum([x[city][carrier] for carrier in bids]) == demands[city]

# 2. The total TEUs assigned to a carrier must not exceed its capacity.
for carrier in bids:
    prob += pulp.lpSum([x[city][carrier] for city in demands]) <= carrier_capacity[carrier]

# 3. Each lane (city) must have at least two carriers assigned.
for city in demands:
    prob += pulp.lpSum([y[city][carrier] for carrier in bids]) >= 2

# 4. Linking constraints for y and x variables
for city in demands:
    for carrier in bids:
        prob += x[city][carrier] >= 5 * y[city][carrier]
        prob += x[city][carrier] <= demands[city] * y[city][carrier] + 4 * (1 - y[city][carrier])

# Solve the problem
prob.solve()

# Extract and display the results
allocated_TEUs = {carrier: sum(x[city][carrier].varValue for city in demands) for carrier in bids}
total_cost = sum(demands[city] * cost_per_TEU[carrier][city] * distances[city] * x[city][carrier].varValue for city in demands for carrier in bids)

print(allocated_TEUs)
print("Total cost:", round(total_cost))


{'Red': 15.0, 'Blue': 21.0, 'Green': 33.0}
Total cost: 11146060


In [5]:
import pulp

# MILP problem formulation
prob = pulp.LpProblem("CarrierAllocationWithTwoCarriers", pulp.LpMinimize)

# Decision Variables
x = pulp.LpVariable.dicts("x", (demands.keys(), bids), lowBound=0, cat="Continuous")
y = pulp.LpVariable.dicts("y", (demands.keys(), bids), cat="Binary")

# Objective Function
prob += pulp.lpSum([x[city][carrier] * cost_per_TEU[carrier][city] * distances[city] for city in demands for carrier in bids])

# Constraints:

# 1. Each city's demand should be satisfied.
for city in demands:
    prob += pulp.lpSum([x[city][carrier] for carrier in bids]) == demands[city]

# 2. If y is 1, x should be at least 5, if y is 0, x should be 0.
for city in demands:
    for carrier in bids:
        prob += x[city][carrier] >= 5 * y[city][carrier]
        prob += x[city][carrier] <= demands[city] * y[city][carrier]

# 3. At least two carriers for each city should have y as 1.
for city in demands:
    prob += pulp.lpSum([y[city][carrier] for carrier in bids]) >= 2

# 4. Each carrier's total loads should not exceed its capacity.
for carrier in bids:
    prob += pulp.lpSum([x[city][carrier] for city in demands]) <= carrier_capacity[carrier]

# Solve the problem
prob.solve()

# Extract and display the results
allocated_TEUs = {carrier: sum(x[city][carrier].varValue for city in demands) for carrier in bids}
total_cost = sum(demands[city] * cost_per_TEU[carrier][city] * distances[city] * x[city][carrier].varValue for city in demands for carrier in bids)

print(allocated_TEUs)
print("Total cost:", round(total_cost))


{'Red': 15.0, 'Blue': 21.0, 'Green': 33.0}
Total cost: 11146060


In [7]:
import pulp
from itertools import combinations

# Generate all combinations of two carriers
carrier_combinations = list(combinations(bids, 2))

# This function sets up and solves the LP for a given combination of carriers for a city
def solve_for_combination(city, combo):
    # Initialize LP problem
    prob = pulp.LpProblem("CarrierAllocationForCombo", pulp.LpMinimize)
    
    # Decision Variables
    x = pulp.LpVariable.dicts("x", (demands.keys(), bids), lowBound=0, cat="Continuous")
    
    # Objective Function
    prob += pulp.lpSum([x[c][carrier] * cost_per_TEU[carrier][c] * distances[c] for c in demands for carrier in bids])
    
    # Constraints
    
    # 1. Each city's demand should be satisfied.
    for c in demands:
        prob += pulp.lpSum([x[c][carrier] for carrier in bids]) == demands[c]
    
    # 2. For the given city and carrier combination, assign at least 5 loads.
    for carrier in combo:
        prob += x[city][carrier] >= 5
        
    # 3. Each carrier's total loads should not exceed its capacity.
    for carrier in bids:
        prob += pulp.lpSum([x[c][carrier] for c in demands]) <= carrier_capacity[carrier]
    
    # Solve the problem
    prob.solve()
    
    # Return the results
    return {
        "allocation": {carrier: sum(x[c][carrier].varValue for c in demands) for carrier in bids},
        "total_cost": sum(demands[c] * cost_per_TEU[carrier][c] * distances[c] * x[c][carrier].varValue for c in demands for carrier in bids)
    }

# Now, for each city, iterate over the combinations and solve the LP
results = []
for city in demands:
    for combo in carrier_combinations:
        result = solve_for_combination(city, combo)
        results.append(result)

# Select the solution with the lowest total cost
best_solution = min(results, key=lambda x: x["total_cost"])

print(best_solution["allocation"])
print("Total cost:", round(best_solution["total_cost"]))


{'Red': 15.0, 'Blue': 21.0, 'Green': 33.0}
Total cost: 10999485


In [10]:
import pulp

# Initialize the MILP problem
prob = pulp.LpProblem("CarrierAllocation", pulp.LpMinimize)

# Decision Variables
x = pulp.LpVariable.dicts("x", (demands.keys(), bids), cat="Binary")
y = pulp.LpVariable.dicts("y", (demands.keys(), bids), lowBound=0, cat="Continuous")

# Objective Function
prob += pulp.lpSum([y[city][carrier] * cost_per_TEU[carrier][city] * distances[city] for city in demands for carrier in bids])

# Constraints

# 1. Each city's demand should be satisfied.
for city in demands:
    prob += pulp.lpSum([y[city][carrier] for carrier in bids]) == demands[city]

# 2. Each carrier's total loads should not exceed its capacity.
for carrier in bids:
    prob += pulp.lpSum([y[city][carrier] for city in demands]) <= carrier_capacity[carrier]

# 3. If a carrier is assigned to a city, it transports at least 5 loads from that city.
for city in demands:
    for carrier in bids:
        prob += y[city][carrier] >= 5 * x[city][carrier]

# 4. At least two carriers are considered for each city.
for city in demands:
    prob += pulp.lpSum([x[city][carrier] for carrier in bids]) >= 2

# 5. Ensure y does not exceed demand if x is 0.
for city in demands:
    for carrier in bids:
        prob += y[city][carrier] <= demands[city] * x[city][carrier]

# Solve the problem
prob.solve()

# Extract and display the results
allocated_TEUs = {carrier: sum(y[city][carrier].varValue for city in demands) for carrier in bids}
allocated_lanes = {carrier: sum(x[city][carrier].varValue for city in demands) for carrier in bids}
total_cost = sum(demands[city] * cost_per_TEU[carrier][city] * distances[city] * y[city][carrier].varValue for city in demands for carrier in bids)

print("Allocated TEUs:", allocated_TEUs)
print("Allocated Lanes:", allocated_lanes)
print("Total cost:", round(total_cost))


Allocated TEUs: {'Red': 15.0, 'Blue': 21.0, 'Green': 33.0}
Allocated Lanes: {'Red': 2.0, 'Blue': 2.0, 'Green': 4.0}
Total cost: 11146060


In [11]:
import pulp

# Initialize the MILP problem
prob = pulp.LpProblem("CarrierAllocation", pulp.LpMinimize)

# Decision Variables
x = pulp.LpVariable.dicts("x", (demands.keys(), bids), cat="Binary")
y = pulp.LpVariable.dicts("y", (demands.keys(), bids), lowBound=0, upBound=None, cat="Continuous")

# Objective Function
prob += pulp.lpSum([y[city][carrier] * cost_per_TEU[carrier][city] * distances[city] for city in demands for carrier in bids])

# Constraints

# 1. Each city's demand should be satisfied.
for city in demands:
    prob += pulp.lpSum([y[city][carrier] for carrier in bids]) == demands[city]

# 2. Each carrier's total loads should not exceed its capacity.
for carrier in bids:
    prob += pulp.lpSum([y[city][carrier] for city in demands]) <= carrier_capacity[carrier]

# 3. Each city should have at least two carriers, and each carrier should have at least 5 TEUs if assigned.
for city in demands:
    prob += pulp.lpSum([x[city][carrier] for carrier in bids]) >= 2
    for carrier in bids:
        prob += y[city][carrier] >= 5 * x[city][carrier]

# 4. If a carrier is not assigned to a city, it should not transport any TEUs from that city.
for city in demands:
    for carrier in bids:
        prob += y[city][carrier] <= demands[city] * x[city][carrier]

# Solve the problem
prob.solve()

# Extract and display the results
allocated_TEUs = {carrier: sum(y[city][carrier].varValue for city in demands) for carrier in bids}
allocated_lanes = {carrier: sum(x[city][carrier].varValue for city in demands) for carrier in bids}
total_cost = sum(demands[city] * cost_per_TEU[carrier][city] * distances[city] * y[city][carrier].varValue for city in demands for carrier in bids)

print("Allocated TEUs:", allocated_TEUs)
print("Allocated Lanes:", allocated_lanes)
print("Total cost:", round(total_cost))


Allocated TEUs: {'Red': 15.0, 'Blue': 21.0, 'Green': 33.0}
Allocated Lanes: {'Red': 2.0, 'Blue': 2.0, 'Green': 4.0}
Total cost: 11146060


In [12]:
# Defining the data:

parts = ["W32A", "Y33A", "W32B", "X33B", "Z32A"]
suppliers = ["Supplier A", "Supplier B", "Supplier C", "Supplier D"]

demands = {
    "W32A": 17000,
    "Y33A": 33000,
    "W32B": 30000,
    "X33B": 30000,
    "Z32A": 4000
}

prices = {
    "W32A": {"Supplier A": 500, "Supplier B": 538, "Supplier C": 570, "Supplier D": 566},
    "Y33A": {"Supplier A": 109, "Supplier B": 108, "Supplier C": 116, "Supplier D": 117},
    "W32B": {"Supplier A": 27, "Supplier B": 28, "Supplier C": 30, "Supplier D": 29},
    "X33B": {"Supplier A": 230, "Supplier B": 225, "Supplier C": 207, "Supplier D": 245},
    "Z32A": {"Supplier A": 324, "Supplier B": 354, "Supplier C": 352, "Supplier D": 378}
}

capacity = {
    "Supplier A": 12600,
    "Supplier B": 26400,
    "Supplier C": 24000,
    "Supplier D": 54000
}

# LP formulation begins

# Define the LP problem
prob = pulp.LpProblem("ProcurementCostMinimization", pulp.LpMinimize)

# Decision variables:
x = pulp.LpVariable.dicts("x", (parts, suppliers), lowBound=0, cat="Continuous")

# Objective function:
prob += pulp.lpSum([x[part][supplier] * prices[part][supplier] for part in parts for supplier in suppliers])

# Constraints:

# 1. Each part's demand should be satisfied.
for part in parts:
    prob += pulp.lpSum([x[part][supplier] for supplier in suppliers]) == demands[part]

# 2. Suppliers' capacity should not be exceeded.
for supplier in suppliers:
    prob += pulp.lpSum([x[part][supplier] for part in parts]) <= capacity[supplier]

# Solve the problem
prob.solve()

# Extract and display the results
allocated_units = {supplier: sum(x[part][supplier].varValue for part in parts) for supplier in suppliers}
total_cost = pulp.value(prob.objective)

allocated_units, total_cost


({'Supplier A': 12600.0,
  'Supplier B': 26400.0,
  'Supplier C': 24000.0,
  'Supplier D': 51000.0},
 21024200.0)

In [13]:
from pulp import LpMinimize, LpProblem, LpVariable, lpSum

# Data
suppliers = ["A", "B", "C", "D"]
parts = ["W32A", "Y33A", "W32B", "X33B", "Z32A"]

# Number of units needed and unit prices
demand = {
    "W32A": 17000,
    "Y33A": 33000,
    "W32B": 30000,
    "X33B": 30000,
    "Z32A": 4000
}

unit_prices = {
    ("W32A", "A"): 500,
    ("W32A", "B"): 538,
    ("W32A", "C"): 570,
    ("W32A", "D"): 566,
    ("Y33A", "A"): 109,
    ("Y33A", "B"): 108,
    ("Y33A", "C"): 116,
    ("Y33A", "D"): 117,
    ("W32B", "A"): 27,
    ("W32B", "B"): 28,
    ("W32B", "C"): 30,
    ("W32B", "D"): 29,
    ("X33B", "A"): 230,
    ("X33B", "B"): 225,
    ("X33B", "C"): 207,
    ("X33B", "D"): 245,
    ("Z32A", "A"): 324,
    ("Z32A", "B"): 354,
    ("Z32A", "C"): 352,
    ("Z32A", "D"): 378
}

# Level of service
service_levels = {
    "A": 92,
    "B": 97,
    "C": 96,
    "D": 95
}

# Cost per % level of service
cost_per_service = {
    "W32A": 10,
    "Y33A": 2,
    "W32B": 1,
    "X33B": 3,
    "Z32A": 8
}

# Create a linear programming problem
prob = LpProblem("GetFit_Procurement", LpMinimize)

# Decision variables
x = LpVariable.dicts("x", [(part, supplier) for part in parts for supplier in suppliers], lowBound=0, cat="Integer")
y = LpVariable.dicts("y", [(part, supplier) for part in parts for supplier in suppliers], lowBound=0, upBound=1, cat="Continuous")

# Objective function
prob += lpSum([x[(part, supplier)] * (unit_prices[(part, supplier)] * demand[part] + 
                                        cost_per_service[part] * (service_levels[supplier] - service_levels["A"])) 
                                        for part in parts for supplier in suppliers])

# Constraints
for part in parts:
    prob += lpSum([x[(part, supplier)] for supplier in suppliers]) == demand[part]

for supplier in suppliers:
    prob += lpSum([x[(part, supplier)] for part in parts]) <= 24000  # Maximum total number of units per supplier

# Solve the problem
prob.solve()

# Print the results
print("Minimum total procurement cost for GetFit:", round(prob.objective.value()))


Minimum total procurement cost for GetFit: 478727672000


In [14]:
from pulp import LpProblem, LpVariable, lpSum, LpMinimize

# Parts and their required quantities
parts = ['W32A', 'Y33A', 'W32B', 'X33B', 'Z32A']
required_quantities = {'W32A': 17000, 'Y33A': 33000, 'W32B': 30000, 'X33B': 30000, 'Z32A': 4000}

# Supplier prices
supplier_prices = {
    'A': {'W32A': 500, 'Y33A': 109, 'W32B': 27, 'X33B': 230, 'Z32A': 324},
    'B': {'W32A': 538, 'Y33A': 108, 'W32B': 28, 'X33B': 225, 'Z32A': 354},
    'C': {'W32A': 570, 'Y33A': 116, 'W32B': 30, 'X33B': 207, 'Z32A': 352},
    'D': {'W32A': 566, 'Y33A': 117, 'W32B': 29, 'X33B': 245, 'Z32A': 378}
}

# Supplier maximum supply limits
supplier_limits = {'A': 12600, 'B': 26400, 'C': 24000, 'D': 54000}

# Supplier E prices
supplier_e_prices = {'W32A': 555, 'Y33A': 120, 'W32B': 35, 'X33B': 255, 'Z32A': 359}

# Total units Supplier E can supply
supplier_e_total_units = 22800

# Create a linear programming problem
problem = LpProblem("GetFit_Procurement", LpMinimize)

# Define decision variables
supplier_vars = {s: {part: LpVariable(f"{s}_{part}", lowBound=0, cat='Integer') for part in parts} for s in supplier_prices}
supplier_e_vars = {part: LpVariable(f"SupplierE_{part}", lowBound=0, cat='Integer') for part in parts}

# Define the objective function (minimize procurement cost)
cost = lpSum(supplier_prices[s][part] * supplier_vars[s][part] for s in supplier_prices for part in parts) \
       + lpSum(supplier_e_prices[part] * supplier_e_vars[part] for part in parts)
problem += cost

# Add constraints for required quantities
for part in parts:
    problem += lpSum(supplier_vars[s][part] for s in supplier_prices) + supplier_e_vars[part] == required_quantities[part]

# Add constraints for supplier limits
for s in supplier_prices:
    problem += lpSum(supplier_vars[s][part] for part in parts) <= supplier_limits[s]

# Add constraint for Supplier E total units
problem += lpSum(supplier_e_vars[part] for part in parts) == supplier_e_total_units

# Solve the problem
problem.solve()

# Get the minimum total procurement cost
min_total_cost = problem.objective.value()
print(f"Minimum Total Procurement Cost: ${min_total_cost:.2f}")


Minimum Total Procurement Cost: $21070000.00


In [15]:
from pulp import LpProblem, LpVariable, lpSum, LpMinimize

# Parts and their required quantities
parts = ['W32A', 'Y33A', 'W32B', 'X33B', 'Z32A']
required_quantities = {'W32A': 17000, 'Y33A': 33000, 'W32B': 30000, 'X33B': 30000, 'Z32A': 4000}

# Supplier prices
supplier_prices = {
    'A': {'W32A': 500, 'Y33A': 109, 'W32B': 27, 'X33B': 230, 'Z32A': 324},
    'B': {'W32A': 538, 'Y33A': 108, 'W32B': 28, 'X33B': 225, 'Z32A': 354},
    'C': {'W32A': 570, 'Y33A': 116, 'W32B': 30, 'X33B': 207, 'Z32A': 352},
    'D': {'W32A': 566, 'Y33A': 117, 'W32B': 29, 'X33B': 245, 'Z32A': 378}
}

# Supplier maximum supply limits
supplier_limits = {'A': 12600, 'B': 26400, 'C': 24000, 'D': 54000}

# Supplier E prices
supplier_e_prices = {'W32A': 555, 'Y33A': 120, 'W32B': 35, 'X33B': 255, 'Z32A': 359}

# Total units Supplier E can supply
supplier_e_total_units = 22800

# Create a linear programming problem
problem = LpProblem("GetFit_Procurement", LpMinimize)

# Define decision variables for regular suppliers
supplier_vars = {s: {part: LpVariable(f"{s}_{part}", lowBound=0, cat='Integer') for part in parts} for s in supplier_prices}

# Define decision variables for Supplier E
supplier_e_vars = {part: LpVariable(f"SupplierE_{part}", lowBound=0, cat='Integer') for part in parts}

# Define the objective function (minimize procurement cost)
cost = lpSum(supplier_prices[s][part] * supplier_vars[s][part] for s in supplier_prices for part in parts) \
       + lpSum(supplier_e_prices[part] * supplier_e_vars[part] for part in parts)
problem += cost

# Add constraints for required quantities
for part in parts:
    problem += lpSum(supplier_vars[s][part] for s in supplier_prices) + supplier_e_vars[part] == required_quantities[part]

# Add constraints for supplier limits
for s in supplier_prices:
    problem += lpSum(supplier_vars[s][part] for part in parts) <= supplier_limits[s]

# Add constraint for Supplier E total units
problem += lpSum(supplier_e_vars[part] for part in parts) == supplier_e_total_units

# Solve the problem
problem.solve()

# Get the minimum total procurement cost
min_total_cost = problem.objective.value()
print(f"Minimum Total Procurement Cost: ${min_total_cost:.2f}")


Minimum Total Procurement Cost: $21070000.00


In [16]:
# Supplier E prices
supplier_e_prices = {'W32A': 555, 'Y33A': 120, 'W32B': 35, 'X33B': 255, 'Z32A': 359}

# Supplier E variables (quantities)
supplier_e_vars = {part: value.varValue for part, value in supplier_e_vars.items()}

# Calculate the revenue for Supplier E
supplier_e_revenue = sum(supplier_e_vars[part] * supplier_e_prices[part] for part in parts)
print(f"Revenue for Supplier E: ${supplier_e_revenue:.2f}")


Revenue for Supplier E: $3539000.00
